# Why is gradient descent so bad at optimizing polynomial regression?# 

Question from Stackexchange

### Libraries

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('seaborn-white')

### Functions 

In [2]:
def costfunction(theta,X,y):
    m = np.size(y)
    theta = theta.reshape(-1,1)
    
    #Cost function in vectorized form
    h = X @ theta
    J = float((1./(2*m)) * (h - y).T @ (h - y));    
    return J;

def gradient_descent(theta,X,y,alpha = 0.0005,num_iters=1000):
    m = np.size(y)
    
    for i in range(num_iters):
        #Grad function in vectorized form
        h = X @ theta
        theta = theta - alpha * (1/m)* (X.T @ (h-y))
        
    return theta

def grad(theta,X,y):
    #Initializations
    theta = theta[:,np.newaxis]
    m = len(y)
    grad = np.zeros(theta.shape)
    
    #Computations
    h = X @ theta
    
    grad = (1./m)*(X.T @ ( h - y))
    
    return (grad.flatten())


def polynomial_features(data, deg):
    data_copy=data.copy()
    
    for i in range(1,deg):
        data_copy['X'+str(i+1)]=data_copy['X'+str(i)]*data_copy['X1']

    return data_copy

### Initializing the data

In [3]:
#Create data from sin function with uniform noise
x = np.linspace(0,1,40)
noise = 1*np.random.uniform(  size = 40)
y = np.sin(x * 1.5 * np.pi ) 
y_noise = (y + noise).reshape(-1,1)
X = np.vstack((np.ones(len(x)),x)).T

degree = 15
X_d = polynomial_features(pd.DataFrame({'X0':1,'X1': x}),degree)

### Closed form solution 

In [4]:
def closed_form_solution(X,y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

closed_form_solution(X_d.values,y_noise)

array([[-5.68976477e-01],
       [ 4.14046200e+01],
       [-9.03017908e+02],
       [ 9.58065441e+03],
       [-4.49088574e+04],
       [ 5.96962228e+04],
       [ 3.18227898e+05],
       [-1.72978262e+06],
       [ 3.86409797e+06],
       [-4.70824016e+06],
       [ 3.23743378e+06],
       [-1.74472645e+06],
       [ 2.06049877e+06],
       [-2.41570547e+06],
       [ 1.40990123e+06],
       [-3.15211551e+05]])

### Numpy only fit

In [5]:
theta_result_1 = gradient_descent(np.zeros((len(X_d.T),1)).reshape(-1,1), X_d,y_noise,alpha = .1,num_iters=1000)
theta_result_1

array([[ 1.0468006 ],
       [ 0.84807577],
       [-0.62152239],
       [-0.91258239],
       [-0.80166835],
       [-0.59678275],
       [-0.3974059 ],
       [-0.23015392],
       [-0.09770137],
       [ 0.00433034],
       [ 0.08165229],
       [ 0.13949769],
       [ 0.18217817],
       [ 0.21309703],
       [ 0.23489246],
       [ 0.24959621]])

### Sciy optimize fit

In [6]:
import scipy.optimize as opt

theta_init = np.ones((len(X_d.T),1)).reshape(-1,1)
model_t = opt.minimize(fun = costfunction, x0 = theta_init , args = (X_d, y_noise), 
                             method = 'BFGS', jac = grad, options={'maxiter':1000})
model_t.x

array([  0.3653751 ,   5.01714559,   2.42495055, -19.64125507,
        -3.33104422,   9.07415879,  10.59592578,   6.11927452,
         0.50261061,  -3.6346289 ,  -5.45126495,  -5.11773491,
        -3.26010397,  -0.63362555,   2.04526255,   4.17234749])

### Sklearn fit 

In [7]:
from sklearn import linear_model
model_d = linear_model.LinearRegression(fit_intercept=False)
model_d.fit(X_d,y_noise)
model_d.coef_

array([[ 3.01597858e-01,  6.83509165e+01, -2.91401167e+03,
         5.22721768e+04, -4.88790193e+05,  2.56258615e+06,
        -6.73068632e+06, -9.34747716e+05,  7.34271140e+07,
        -2.83379636e+08,  6.03268254e+08, -8.23368800e+08,
         7.38684424e+08, -4.22986907e+08,  1.40557246e+08,
        -2.06594837e+07]])

### Statsmodel fit 

In [8]:
import statsmodels.api as sm
model_sm = sm.OLS(y_noise, X_d)
res = model_sm.fit()
print(res.summary())

/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     14.82
Date:                Wed, 06 Jun 2018   Prob (F-statistic):           1.23e-08
Time:                        21:11:37   Log-Likelihood:                 2.6712
No. Observations:                  40   AIC:                             26.66
Df Residuals:                      24   BIC:                             53.68
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
X0             0.3016      0.292      1.033      0.3